Sales_Forecasting.ipynb

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
import joblib
import pickle
import os

path = os.getcwd()

Load Data

In [ ]:
df = pd.read_csv(f'{path}/data/sales_data.csv', parse_dates=['date'])
df = df.sort_values('date')

Handle Missing Values

In [ ]:
df.fillna(method='ffill', inplace=True)

Feature Engineering

In [ ]:
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['sales_lag_1'] = df['sales'].shift(1)
df['sales_lag_7'] = df['sales'].shift(7)
df['sales_lag_30'] = df['sales'].shift(30)
df['sales_roll_mean_7'] = df['sales'].rolling(window=7).mean()
df['sales_roll_std_7'] = df['sales'].rolling(window=7).std()
df.dropna(inplace=True)

Features and Target

In [ ]:
X = df.drop(['date', 'sales'], axis=1)
y = df['sales']

Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Save Scaler

In [ ]:
joblib.dump(scaler, f'{path}/models/scaler.pkl')

XGBoost Model

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f'XGBoost RMSE: {rmse_xgb}')
joblib.dump(xgb_model, f'{path}/models/xgboost_model.pkl')

ARIMA Model

In [ ]:
arima_order = (5, 1, 0)
train_size = int(len(df) * 0.8)
train, test = df['sales'][:train_size], df['sales'][train_size:]
arima_model = ARIMA(train, order=arima_order)
arima_result = arima_model.fit()
forecast = arima_result.forecast(steps=len(test))
rmse_arima = np.sqrt(mean_squared_error(test, forecast))
print(f'ARIMA RMSE: {rmse_arima}')
with open(f'{path}/models/arima_model.pkl', 'wb') as pkl:
    pickle.dump(arima_result, pkl)

Visualization

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=df['date'], y=df['sales'], label='Actual Sales')
plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

Feature Importance

In [ ]:
feature_importances = pd.Series(xgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=feature_importances.values, y=feature_importances.index)
plt.title('Feature Importances (XGBoost)')
plt.show()

Forecasting with Selected Model

In [ ]:
selected_model = 'XGBoost' if rmse_xgb < rmse_arima else 'ARIMA'
print(f'Selected Model: {selected_model}')

In [ ]:
if selected_model == 'XGBoost':
    latest_data = X.tail(1)
    latest_data_scaled = scaler.transform(latest_data)
    forecast_value = xgb_model.predict(latest_data_scaled)[0]
    print(f'Next Day Forecasted Sales (XGBoost): {forecast_value:.2f}')
else:
    forecast_value = arima_result.forecast(steps=1)[0]
    print(f'Next Day Forecasted Sales (ARIMA): {forecast_value:.2f}')